In [1]:
import skforecast
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from dateutil.relativedelta import relativedelta

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster

c:\OneDrive - TRIMEDX Holdings, LLC\Miniconda3\envs\ff_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('database/DIM_PLAYER_GAMES.csv')
test = list(df['PLAYER'].unique()[:5])
test

['Christian McCaffrey',
 'Tyreek Hill',
 'CeeDee Lamb',
 'Josh Allen',
 'Raheem Mostert']

In [3]:
def time_series_convert(player: str):
    df = pd.read_csv('database/DIM_PLAYER_GAMES.csv')
    data = df.loc[df['PLAYER']==player]
    data['FORECAST_DATE'] = pd.date_range("2018-01-01", periods=len(data), freq="MS")
    data['FORECAST_DATE'] = pd.to_datetime(data['FORECAST_DATE'], format='%Y-%m-%d')
    data = data.set_index('FORECAST_DATE')
    data = data.asfreq('MS')
    data = data['PPR']
    return data

In [4]:
df_multi = pd.DataFrame(test, columns=['PLAYER'])
df_multi['data'] = df_multi['PLAYER'].map(lambda x :time_series_convert(x))
df_multi

C:\Users\012356.CORPORATE\AppData\Local\Temp\ipykernel_27600\923225185.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['FORECAST_DATE'] = pd.date_range("2018-01-01", periods=len(data), freq="MS")
C:\Users\012356.CORPORATE\AppData\Local\Temp\ipykernel_27600\923225185.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['FORECAST_DATE'] = pd.to_datetime(data['FORECAST_DATE'], format='%Y-%m-%d')
C:\Users\012356.CORPORATE\AppData\Local\Temp\ipykernel_27600\923225185.py:4: SettingWithCopyWarning: 
A

,PLAYER,data
0,Christian McCaffrey,FORECAST_DATE 2018-01-01 15.5 2018-02-01 ...
1,Tyreek Hill,FORECAST_DATE 2018-01-01 36.3 2018-02-01 ...
2,CeeDee Lamb,FORECAST_DATE 2018-01-01 10.9 2018-02-01 ...
3,Josh Allen,FORECAST_DATE 2018-01-01 17.00 2018-02-01 ...
4,Raheem Mostert,FORECAST_DATE 2018-01-01 25.1 2018-02-01 ...


In [5]:
forecaster = ForecasterAutoreg(
                 regressor = LinearRegression(),
                 lags      = 17
             )


In [6]:
df_multi['forecaster'] = df_multi['data'].map(lambda x : forecaster.fit(y=x.loc[:max(x.index) - relativedelta(months=18)]))
df_multi


ValueError: The maximum lag (17) must be less than the length of the series (17).